In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    r'C:\Users\Deju\OneDrive\Documents\facial\emotion\dataset',
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

test_data = datagen.flow_from_directory(
    r'C:\Users\Deju\OneDrive\Documents\facial\emotion\dataset',
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

base_model = InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(299, 299, 3)
)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)

x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(train_data, epochs=15, validation_data=test_data)

model.save("inceptionv3_model.h5")
print("InceptionV3 model saved")


Found 2202 images belonging to 2 classes.
Found 550 images belonging to 2 classes.
Epoch 1/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 997s 14s/step - accuracy: 0.6563 - loss: 0.8958 - val_accuracy: 0.7291 - val_loss: 0.6116
Epoch 2/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.7332 - loss: 0.5818 

In [5]:
import cv2
import numpy as np
import tensorflow as tf

# Load trained model
model = tf.keras.models.load_model("inceptionv3_model.h5")

# Emotion labels (CHANGE according to your dataset)
labels = ["angry","disgusted","fearful","happy","neutral","sad","surprised"]  
# Example: ["Sad", "Happy"]

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame to model input size
    img = cv2.resize(frame, (299, 299))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)

    # Predict emotion
    prediction = model.predict(img)[0][0]

    if prediction > 0.5:
        emotion = labels[1]
        confidence = prediction
    else:
        emotion = labels[0]
        confidence = 1 - prediction

    # Display result
    text = f"{emotion} ({confidence*100:.2f}%)"
    cv2.putText(frame, text, (30, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 255, 0), 2)

    cv2.imshow("Live Emotion Detection", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 